In [2]:
import json
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import SimpleRNN, GRU, LSTM
from keras.layers.core import Dense, Dropout
from keras.layers.wrappers import TimeDistributed
from keras.layers import Convolution1D, MaxPooling1D
from keras.utils import np_utils
import itertools
import ast
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import sys, os
import traceback
from itertools import chain
import numpy as np
import pandas as pd

import tensorflow as tf
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

import matplotlib.pyplot as plt

%matplotlib inline

Using TensorFlow backend.


In [3]:
def get_sentences(data_list):
    sentences = []
    for a in data_list.article.unique():
        words = []
        prev_n = 0
        for _,s in data_list.loc[data_list.article == a].iterrows():
            n = s['sentence_no']
            if n != prev_n:
                if len(words) > 2 and len(set([x[1] for x in words])) > 1:
                    sentences.append(words)
                words = []
            words.append((s['word'], s['encoding'], s['part_of_speech']))
            prev_n = n
            
        if len(words) > 2 and len(set([x[1] for x in words])) > 1:
            sentences.append(words)
            
    return sentences

In [4]:
data_list = pd.read_csv('../data/go/go_cc3.csv')
data_list.word.fillna('NA', inplace=True)

temp = pd.read_csv('../data/go/go_bp3.csv')
temp.word.fillna('NA', inplace=True)

temp1 = pd.read_csv('../data/go/go_mf3.csv')
temp1.word.fillna('NA', inplace=True)
temp1 = temp1.rename(columns={'encoding' : 'encoding_z'})

data_list = data_list.merge(temp, on=['article', 'sentence_no', 'word_no'])
data_list = data_list.merge(temp1, on=['article', 'sentence_no', 'word_no'])
data_list.sort_values(['article', 'sentence_no'], kind='mergesort', inplace=True)
x_counts = data_list.encoding_x.value_counts()
y_counts = data_list.encoding_y.value_counts()
z_counts = data_list.encoding_z.value_counts()
# data_list['encoding_x'] = data_list.encoding_x.apply(lambda x: x if x_counts[x] > 4 else 'O')
# data_list['encoding_y'] = data_list.encoding_y.apply(lambda x: x if y_counts[x] > 4 else 'O')
# data_list['encoding'] = [i if (i == 'O' and j == 'O') else j for i,j in zip(data_list.encoding_x, data_list.encoding_y)]
temp_encoding = []
for i,j,k in zip(data_list.encoding_x, data_list.encoding_y, data_list.encoding_z):
    if i == j and j == k:
        temp_encoding.append('O')
    elif i == 'O' and j != 'O':
        temp_encoding.append(j)
    elif j == 'O' and i != 'O':
        temp_encoding.append(i)
    else:
        temp_encoding.append(k)
data_list['encoding'] = temp_encoding
data_list.drop(['word_x', 'word_y', 'part_of_speech_x', 'part_of_speech_y',
                'encoding_x', 'encoding_y', 'encoding_z'], axis=1, inplace=True)
data_list.set_index(['article', 'sentence_no'], inplace=True)
val_cnt = data_list.encoding.value_counts()
# data_list['encoding'] = data_list.encoding.apply(lambda x: 'O' if val_cnt[x] < 5 else x)
# data_list = data_list.rename(columns={'word_x' : 'word'})

In [5]:
go = pd.read_csv('../data/go/GO_AllSubsumers.tsv', sep='\t', header=None)
go.columns = ['Child', 'Parent']
go.Child = go.Child.apply(lambda x: x.replace('_',':'))
go.Parent = go.Parent.apply(lambda x: x.replace('_',':'))
go_vc = go.Parent.value_counts()
go['ParentCount'] = go.Parent.apply(lambda x: go_vc[x])
go = go.sort_values('ParentCount', ascending=False)
go

,Child,Parent,ParentCount
117551,GO:0003946,owl:Thing,4941
6574,GO:0034259,owl:Thing,4941
47219,GO:0001610,owl:Thing,4941
47222,GO:0001611,owl:Thing,4941
47269,GO:0001606,owl:Thing,4941
47270,GO:0001603,owl:Thing,4941
47276,GO:0001600,owl:Thing,4941
47359,NCBITaxon:61172,owl:Thing,4941
6605,GO:0034273,owl:Thing,4941
47639,GO:0035031,owl:Thing,4941


In [26]:
cnts = val_cnt.copy()
data_list['new_encoding'] = data_list.encoding

In [27]:
r = 0
while r < 3:
    news = []
    for _, w in data_list.iterrows():
        enc = w['new_encoding']
        if cnts[enc] < 5:
            pre = ''
            if enc[1] == '_':
                pre = enc[0:2]
                enc = enc[2:]
            vals = go[go.Child == enc].Parent.values
            if len(vals) == 0 or not vals[0].startswith("GO:"):
                news.append(pre + enc)
                continue
            new_enc = pre + vals[0]
            news.append(new_enc)
        else:
            news.append(enc)
    cnts = pd.Series(news).value_counts()
    data_list['new_encoding'] = news
    print(len(cnts[cnts < 5]))
    r += 1

835
495
273


In [26]:
data_list[data_list.encoding != data_list.new_encoding].new_encoding.value_counts()

1_GO:0048856    25
2_GO:0032991    24
2_GO:0048856    23
1_GO:0032991    21
3_GO:0097458    14
1_owl:Thing     13
3_GO:0032991    12
1_GO:0097458    12
2_GO:0097458    12
1_GO:0005575    11
2_GO:0009987    11
1_GO:0001708    11
2_GO:0005575    11
GO:0032991      10
1_GO:0008150    10
2_GO:0008150    10
1_GO:0050794     9
3_GO:0048869     9
1_GO:0032501     9
1_GO:0048869     9
1_GO:0009987     9
2_GO:0048732     9
GO:0097458       9
2_GO:0032501     9
GO:0032501       8
2_GO:0009888     8
2_GO:0001708     8
2_GO:0048869     8
2_GO:0006810     8
GO:0022403       8
                ..
2_GO:0043256     1
1_GO:0031968     1
1_GO:0099024     1
1_GO:0032446     1
1_GO:0030312     1
GO:0043231       1
2_GO:0044456     1
1_GO:0006915     1
1_GO:0034968     1
2_GO:0044422     1
1_GO:0075259     1
1_GO:0006914     1
2_GO:0009913     1
3_GO:0000152     1
5_GO:0001708     1
1_GO:0051649     1
1_GO:0042981     1
4_GO:0050877     1
2_GO:0045123     1
2_GO:0007276     1
2_GO:0051604     1
2_GO:0048812

In [31]:
data_list[data_list.new_encoding == '1_GO:0048856'].word.value_counts()

development                     4
formation                       3
hair                            3
nephron                         3
blood                           2
pharyngeal                      2
otolith                         1
forebrain                       1
prosencephalon-mesencephalon    1
head                            1
lung                            1
multicellular                   1
Nephron                         1
inner                           1
Name: word, dtype: int64